## EPS/ESE 135: Observing the Ocean
### Data Analysis Assignment 5 (Intro): Extreme events

*You don't have to run any code in this notebook, but I have provided suggested Python approaches to the exercises, so please read to the end.*

For this assignment you will be analyzing a tide gauge record from Newport, Rhode Island. This record comes from a [University of Hawaii Sea Level Center (UHSLC) database](https://uhslc.soest.hawaii.edu/data/) that archives hourly and daily data for certain tide gauge stations. In contrast to the data from PSMSL that you worked with last week, which were averaged monthly, the higher temporal resolution of the UHSLC data will allow us to look at individual extreme events.

This assignment is based on the first two tutorials of the [ClimateMatch Academy](https://comptools.climatematch.io/tutorials/intro.html) materials on [Extremes and Variability](https://comptools.climatematch.io/tutorials/W2D3_ExtremesandVariability/student/W2D3_Intro.html). If you are looking for more details or background information take a look at the materials on their website, which is a really great resource for climate data analysis.

### Defining an extreme event

When you plot the Newport sea level record, you'll see that there is a huge amount of variability (including some pretty large tides). You can quickly use pandas to generate a list of summary statistics for a dataframe `df` using the function `df.describe()`.

However, if we are interested in high sea level events, we can first extract the maximum recorded sea level for each year and then look at the statistics of those extreme values.

This will allow us to compute **return periods**, or the expected number of years between events of a certain magnitude. This is one way that scientists and engineers define e.g. a 50-year flood.

### Calculating return periods

Start by sorting the annual maxima from highest to lowest. These will be called the **return levels** (i.e. the magnitudes of the extreme events). Then assign each one a rank $r$, with $r=1$ corresponding to the highest value and $r=N$ to the lowest value.

The probability of exceeding any value in a given year $P$ can be calculated as $P=r/(N+1)$.

The corresponding return period $T$ is the inverse of the exceedance probability, $T = 1/P$. (In other words, the lower the probability of an event, the higher the predicted number of years between events of that magnitude.)

Clearly, this does **not** mean that it's impossible for an event to occur more (or less) frequently than its predicted return period! It just means that, based on this record, this is one estimate of the probability of such an event happening in a given year.

### Python tools for this assignment

There is some overlap with the last assignment, with a few new tricks thrown in. My goal here is to show you all of the additional tools you will need to do this assignment but you should implement them yourself! Give your variables sensible names and add your own comments, etc.

As a general rule, you should always be importing all of the packages you need to run your scripts at the top; there's no need to import them again in subsequent cells.

In [ ]:
# import python modules for assignment 5

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

The csv file has 5 columns: year, month, day, hour, and relative sea level in millimeters. There are no headers so you should add them manually when you read the csv into pandas.

![screenshot of csv file](csvscreenshot.png)

In [ ]:
# you decide what to name your dataframe and insert the correct filename
df0 = pd.read_csv(filename,names=['year','month','day','hour','SL'])

# the fill value in this dataset is -32767 for some reason so let's cut
#    those out:
df1 = df0[df0['SL']>-30000]

As we've run into a few times, different ways of expressing dates/time can have different benefits. In this assignment, you will use both: datetime because it makes it easy to group data with pandas, and decimal years because they are convenient for calculating trends.

In [ ]:
# add a column with the datetime 
df1['datetime'] = pd.to_datetime(df1[["year", "month", "day", "hour"]])
df1.set_index('datetime') # tells pandas to use datetime as the index

# now use datetime to calculate decimal years:
# total_seconds is a built-in function that will calculate seconds elapsed
#     between datetime values
x_sec = (df1.index - df1.index[0]).total_seconds()

# how many seconds per year?
# 365.25 days/year
# 24 hours/day...
seconds_per_year = ...

# calculate decimal years
dec_year = x_sec/seconds_per_year

Note that this gives you decimal years starting from 0 (rather than from the middle of 1930); you will use this to calculate a linear trend so it's okay for this purpose if they are offset.

Since your dataframe is indexed by datetime, you can simply use `resample` to get the annual maxima that you will use for the statistical calculations.

In [ ]:
# please name this dataframe something descriptive
df2 = df1.resample("YE").max()

# at this point you could also clean things up by dropping extra columns
df2 = df2[["SL"]].copy()

You can easily quickly plot a histogram of any dataframe using `df['variable'].hist()` and you can add a vertical line to a plot using `plt.axvline(x)`. You can add a `label='description'` argument to each of those if you are going to add a legend to your plot.

**Calculating return periods**

If there are `nan` (not-a-number) values in your dataframe, Python doesn't know how to rank those, so you need to remove them first.

Use [np.sort()](https://numpy.org/doc/stable/reference/generated/numpy.sort.html) to sort the data. This will sort in ascending order (lowest to highest), so you can reverse the order using an [indexing trick](https://www.geeksforgeeks.org/python/how-to-reverse-a-list-in-python-using-slicing/). 

Then use [stats.rankdata()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.rankdata.html) to rank the maxima. Again, by default this function ranks values from lowest to highest. Here you can flip the order by multiplying by negative 1.

In [ ]:
# drop NaN rows from dataframe
df3 =  df2.dropna()

# create a numpy array of return levels sorted from highest to lowest
np1 = np.sort(df3['SL'])[::-1]

# does it look right?
np1

In [ ]:
# use scipy to rank the return levels
np2 = stats.rankdata(np1*-1)

# does it look right? 
# (rank 1 should correspond to the highest value of the sorted list and
#    if there are any duplicate values in the sorted list they should have
#    the same rank here)
np2

Now you can do the calculations given above for exceedance probability $P$ and return period $T$.

In [ ]:
# number of years of data
N = len(np1)

# calculation for exceedance probability
P = ...

# calculation for return period 
T = ...

To make a scatter plot with a log scale on the x-axis:

In [ ]:
#create axis object
fig, ax = plt.subplots()

# plot data on axis
ax.plot(x_variable, y_variable, "o")

# choose x-axis scale:
ax.set_xscale("log")

Keep adding appropriate axis labels, titles, and line labels/legends to your plots!